In [4]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data, get_dim_data, ROOT_DIR
from pyspark.sql import functions as f
spark = get_spark_session("small_files")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

dim_taxi_zones, dim_rates, dim_payments, dim_vendor = get_dim_data(spark)

output_dir = ROOT_DIR + "output/yellow_taxi.parquet"

In [3]:
yellow_taxi_data.repartition(1000).write.mode('overwrite').parquet(output_dir)

In [5]:
yellow_taxi_small_files = spark.read.parquet(output_dir)

In [8]:
%timeit yellow_taxi_data.count()

31.1 ms ± 3.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit yellow_taxi_small_files.count()

427 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Back in 2020 I had a lot of issues with different situations, some of them are still relevant today. Here is my blog post from [back then](https://uncledata.substack.com/p/4-simple-tips-to-improve-your-apache).

Broadcast join we've covered, repartition and coalesce in a way is solved with AQE, but still sometimes helps if you want to have more control.


In [17]:
del spark

**Use openCostInBytes size which is closer to your small file sizes. This will be more efficient!**

In [18]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .appName("small_files")
         .config("spark.files.openCostInBytes", 140046)
         .master("local[*]")
         .getOrCreate())

In [19]:
%timeit yellow_taxi_small_files.count()

391 ms ± 9.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Rule of Thumb is that a partition should be ± 128MB (good range is 80-200MB)